In [1]:
import pandas as pd
import numpy as np
import json
data = pd.read_csv("../data/output_data.csv")


In [2]:
data = data.rename(columns={'0': 'Transcript_ID', '1': 'Position',"2":"Base_seq","3":"Sample_reads"})

In [3]:
data_info = pd.read_csv("../data/data.info")


In [4]:
# Mean and std
def feature_engin1(rows):
    sample_reads = json.loads(rows["Sample_reads"])
    output = {"Mean":[],"Standard_deviation":[]}
    matrix = np.array(sample_reads)
    column_means = np.mean(matrix, axis=0)
    column_std = np.std(matrix,axis=0)
    output = [list(column_means),list(column_std)]
    return output

In [5]:
data["Compiled_reads"] = data.apply(feature_engin1,axis=1)
#json.loads(data.iloc[0]["Sample_reads"])

In [6]:
data.head()

,Transcript_ID,Position,Base_seq,Sample_reads,Compiled_reads
0,ENST00000000233,244,AAGACCA,"[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0...","[[0.008264378378378385, 4.223783783783786, 123..."
1,ENST00000000233,261,CAAACTG,"[[0.0126, 1.95, 111.0, 0.0125, 1.27, 108.0, 0....","[[0.006609244186046515, 3.2164244186046504, 10..."
2,ENST00000000233,316,GAAACAG,"[[0.00432, 2.02, 104.0, 0.00299, 3.56, 99.3, 0...","[[0.0075699999999999995, 2.94054054054054, 105..."
3,ENST00000000233,332,AGAACAT,"[[0.0134, 4.71, 132.0, 0.00447, 4.24, 98.8, 0....","[[0.010620250000000005, 6.4763499999999965, 12..."
4,ENST00000000233,368,AGGACAA,"[[0.015, 6.97, 118.0, 0.0106, 3.04, 123.0, 0.0...","[[0.010700505050505058, 6.415050505050505, 117..."


In [8]:
data = data[['Transcript_ID', 'Position', 'Base_seq', 'Compiled_reads']]

In [9]:
data.head()

,Transcript_ID,Position,Base_seq,Compiled_reads
0,ENST00000000233,244,AAGACCA,"[[0.008264378378378385, 4.223783783783786, 123..."
1,ENST00000000233,261,CAAACTG,"[[0.006609244186046515, 3.2164244186046504, 10..."
2,ENST00000000233,316,GAAACAG,"[[0.0075699999999999995, 2.94054054054054, 105..."
3,ENST00000000233,332,AGAACAT,"[[0.010620250000000005, 6.4763499999999965, 12..."
4,ENST00000000233,368,AGGACAA,"[[0.010700505050505058, 6.415050505050505, 117..."


In [21]:
# Use apply and pd.Series to split the lists into separate columns
new_columns = data['Compiled_reads'].apply(pd.Series).apply(pd.Series)
new_columns = pd.concat([new_columns[0].apply(pd.Series), new_columns[1].apply(pd.Series)], axis = 1)


new_columns.columns = ['mean1', 'mean2', 'mean3', 'mean4', 'mean5', 'mean6', 'mean7', 'mean8', 'mean9',
              'sd1', 'sd2', 'sd3', 'sd4', 'sd5', 'sd6', 'sd7', 'sd8', 'sd9']

# Combine the new columns with the original DataFrame
data = pd.concat([data, new_columns], axis=1)

# Drop the original 'your_column' if you no longer need it
data = data.drop('Compiled_reads', axis=1)

# Display the resulting DataFrame
data.head()


,Transcript_ID,Position,Base_seq,mean1,mean2,mean3,mean4,mean5,mean6,mean7,...,mean9,sd1,sd2,sd3,sd4,sd5,sd6,sd7,sd8,sd9
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,80.570270,0.005384,1.843025,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,94.290698,0.003588,1.689167,2.981182,0.003767,1.237045,3.526528,0.004528,1.466295,2.492530
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,89.364324,0.004444,1.311103,1.492577,0.005035,0.893481,3.203816,0.004497,0.800496,1.484341
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006102,...,89.154000,0.006121,2.459317,3.202963,0.004612,1.393868,1.928009,0.003595,0.741771,3.494937
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,85.178788,0.007151,2.743228,2.664643,0.005407,1.753894,2.177236,0.005908,1.341822,2.379045


In [22]:
data_info.head()
data_info = data_info.rename(columns={'transcript_id': 'Transcript_ID'})

In [23]:
merged_right = pd.merge(data_info, data, on='Transcript_ID', how='right')
gene_ids = list(set(merged_right.gene_id))
len(gene_ids)

3852

In [24]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(gene_ids,test_size=0.3,random_state = 40)

In [25]:
train_data = merged_right[merged_right["gene_id"].isin(train)]
test_data = merged_right[merged_right["gene_id"].isin(test)]

In [26]:
len(train_data)
len(test_data)/len(merged_right)

0.31875393881598035

In [30]:
train_data.head()

,gene_id,Transcript_ID,transcript_position,label,Position,Base_seq,mean1,mean2,mean3,mean4,...,mean9,sd1,sd2,sd3,sd4,sd5,sd6,sd7,sd8,sd9
0,ENSG00000004059,ENST00000000233,244,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,...,80.57027,0.005384,1.843025,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168
1,ENSG00000004059,ENST00000000233,261,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,...,80.57027,0.005384,1.843025,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168
2,ENSG00000004059,ENST00000000233,316,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,...,80.57027,0.005384,1.843025,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168
3,ENSG00000004059,ENST00000000233,332,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,...,80.57027,0.005384,1.843025,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168
4,ENSG00000004059,ENST00000000233,368,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,...,80.57027,0.005384,1.843025,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


X_train = train_data[['mean1', 'mean2', 'mean3', 'mean4', 'mean5', 'mean6', 'mean7', 'mean8', 'mean9', 'sd1', 'sd2', 'sd3', 'sd4', 'sd5', 'sd6', 'sd7', 'sd8', 'sd9']]
Y_train = train_data['label']
X_test = test_data[['mean1', 'mean2', 'mean3', 'mean4', 'mean5', 'mean6', 'mean7', 'mean8', 'mean9', 'sd1', 'sd2', 'sd3', 'sd4', 'sd5', 'sd6', 'sd7', 'sd8', 'sd9']]
Y_test = test_data['label']

In [32]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [33]:
# Make predictions on the test data
Y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 95.64%


In [ ]:
import joblib
joblib.dump(rf_classifier, './model/random_forest_model.pkl')